In [1]:
from model import *
import re
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
from matplotlib import pyplot as plt
import yaml
import pickle
# Gaoyj1019

In [2]:
with open('testdata.txt') as f:
    sentences = f.readlines()
# print(sentences)
MAX_SENT_LEN = 10


In [3]:
with open("input_vocab", "rb") as fp:   # Unpickling
    eng_sentences = pickle.load(fp)
with open("target_vocab", "rb") as fp:   # Unpickling
    deu_sentences = pickle.load(fp)
    
# eng_words.update(eng_sent)
  # deu_words.update(deu_sent)
with open("input_words", "rb") as fp:   # Unpickling
    eng_words = pickle.load(fp)
with open("target_words", "rb") as fp:   # Unpickling
    deu_words = pickle.load(fp)


In [4]:
ENG_VOCAB_SIZE = len(eng_words)
DEU_VOCAB_SIZE = len(deu_words)
NUM_EPOCHS = 20
HIDDEN_SIZE = 16
EMBEDDING_DIM = 30
BATCH_SIZE = 128
NUM_HEADS = 2
NUM_LAYERS = 3
LEARNING_RATE = 1e-3
DROPOUT = .3
DEVICE = torch.device('cpu') 

In [5]:
print(ENG_VOCAB_SIZE, DEU_VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_SIZE, NUM_HEADS, NUM_LAYERS, MAX_SENT_LEN, MAX_SENT_LEN, DROPOUT)

model = TransformerNet(ENG_VOCAB_SIZE, DEU_VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_SIZE, NUM_HEADS, NUM_LAYERS, MAX_SENT_LEN, MAX_SENT_LEN, DROPOUT).to(DEVICE)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
model.load_state_dict(torch.load('mymodel.mp'))
model.eval()

6956 4599 30 16 2 3 10 10 0.3


/home/postgrads/2650807G/spack/opt/spack/linux-centos7-x86_64_v3/gcc-4.8.5/anaconda3-2022.10-ldbv2ghkumdvkfs6y6xxwrtticnzqxhf/envs/BioTokens-cpu/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


TransformerNet(
  (enc_embedding): Embedding(6956, 30)
  (dec_embedding): Embedding(4599, 30)
  (enc_pe): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_pe): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=30, out_features=30, bias=True)
        )
        (linear1): Linear(in_features=30, out_features=16, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (linear2): Linear(in_features=16, out_features=30, bias=True)
        (norm1): LayerNorm((30,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((30,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.3, inplace=False)
        (dropout2): Dropout(p=0.3, inplace=False)
      )
    )
  )
  (decoder): TransformerDecoder(
    (layers): 

In [8]:
outputs = model(torch.tensor(eng_sentences[4],dtype=torch.int).reshape(1,-1),torch.tensor(deu_sentences[3],dtype=torch.int).reshape(1,-1))
_, y_pred = torch.max(outputs.permute(1,2,0).data, 1)

print(y_pred)
print(torch.tensor(deu_sentences[4],dtype=torch.int).reshape(1,-1))

input_x =  [eng_words[k] for k in torch.tensor(eng_sentences[4],dtype=torch.int).reshape(1,-1)[0]]
target_y = [deu_words[k] for k in torch.tensor(deu_sentences[3],dtype=torch.int).reshape(1,-1)[0]]
output_y = [deu_words[k] for k in y_pred[0]]
print(input_x)
print(target_y)
print(output_y)

tensor([[4280, 1939,  428, 1925, 3598, 3078, 4588, 2170, 2170, 3071]])
tensor([[4280, 2637, 2585, 1271, 3973, 2460, 4222, 2047, 2611, 4588]],
       dtype=torch.int32)
['<sos>', 'wenn', 'du', 'so', 'müde', 'bist', 'geh', 'ins', 'bett', '<eos>']
['<sos>', 'i', 'm', 'very', 'angry', 'now', '<eos>', '<pad>', '<pad>', '<pad>']
['<sos>', 'i', 'm', 'very', 'with', 'now', '<eos>', '<pad>', '<pad>', 'trout']
